In [244]:
import spacy

example = 'I saw the man with a telescope.'
spacy_nlp = spacy.load('en_core_web_sm')

# First Answer:
The function return the path of dependencies from the ROOT to the token, For doing this i used the **ancestors property** of the token.

In [245]:
def depPath(sentence):
  doc = spacy_nlp(sentence)
  results = {}
  for token in doc:
      results[token.text] = [*reversed(list([anc.dep_ + ':' + anc.text for anc in token.ancestors])), token.dep_ + ':' + token.text]
  return results

In [246]:
res = depPath(example)
for key in res:
  print(f"{key}: {res[key]}")

I: ['ROOT:saw', 'nsubj:I']
saw: ['ROOT:saw']
the: ['ROOT:saw', 'dobj:man', 'det:the']
man: ['ROOT:saw', 'dobj:man']
with: ['ROOT:saw', 'dobj:man', 'prep:with']
a: ['ROOT:saw', 'dobj:man', 'prep:with', 'pobj:telescope', 'det:a']
telescope: ['ROOT:saw', 'dobj:man', 'prep:with', 'pobj:telescope']
.: ['ROOT:saw', 'punct:.']


# Second Answer:
This function uses the **subtree property** of the tokens. That is sequence containing the token and all the token’s syntactic descendants.

In [247]:
def subTreeFromToken(sentence):
  doc = spacy_nlp(sentence)
  return {token: list(token.subtree) for token in doc}

In [248]:
subTreeFromToken(example)

{I: [I],
 saw: [I, saw, the, man, with, a, telescope, .],
 the: [the],
 man: [the, man, with, a, telescope],
 with: [with, a, telescope],
 a: [a],
 telescope: [a, telescope],
 .: [.]}

# Third answer:
Using the **subtree property** of each tokens check if the given list of tokens is a valid subtree.


In [249]:
def checkListIsSubtree(sentence, possibleSubTree):
  doc = spacy_nlp(sentence)
  for token in doc:
    if [t.text for t in token.subtree] == possibleSubTree:
      return True
  return False

In [250]:
listw = [['I','saw'],['with', 'a', 'telescope'],['a', 'telescope']]
for pSubT in listw:
  print(checkListIsSubtree(example,pSubT))

False
True
True


# Fourth answer:


In [251]:
def getSpanHead(sentence):
  doc = spacy_nlp(sentence)
  return doc[:].root

In [252]:
examples = [example,'I got my new car','Angela Merkel from Germany and the American Donald Trump dont share many opinions']
for e in examples:
  print(f"{e}: {getSpanHead(e)}")

I saw the man with a telescope.: saw
I got my new car: got
Angela Merkel from Germany and the American Donald Trump dont share many opinions: share


# Fifth answer
The function takes as input a sentence and returns a dictionary containing as key the relation (**'nsubj', 'dobj', 'iobj'**) and as value a list containing the words related to the key. In order to do that the sentence is parsed and for every token, if the dependency relation is one of the key above, a list containing the subtree of the token is populated.
**Indirect object** relations are represented by the **dative** keyword, since the **iobj** keyword is deprecated since in the latest versions of spaCy.

In [253]:
def sentenceObjSpans(sentence):
  doc = spacy_nlp(sentence)
  dct = {'nsubj':[],'dobj':[], 'dative':[]}
  for token in doc:
    if token.dep_ == 'nsubj' or token.dep_ == 'dobj' or token.dep_ == 'dative':
      for anc in token.subtree:
        dct[token.dep_].append(anc.text)
  return dct

In [254]:
for e in examples:
  print(sentenceObjSpans(e))

{'nsubj': ['I'], 'dobj': ['the', 'man', 'with', 'a', 'telescope'], 'dative': []}
{'nsubj': ['I'], 'dobj': ['my', 'new', 'car'], 'dative': []}
{'nsubj': ['Angela', 'Merkel', 'from', 'Germany', 'and', 'the', 'American', 'Donald', 'Trump'], 'dobj': ['many', 'opinions'], 'dative': []}
